In [12]:
from nvidia.dali import pipeline_def
import nvidia.dali.fn as fn
import nvidia.dali.types as types
import nvidia.dali as dali
import librosa as librosa
import librosa.display
import numpy as np
import os
import numpy
import random
import torch
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline  

In [7]:
rng_seed = 1234
random.seed(rng_seed)
np.random.seed(rng_seed)
os.environ['PYTHONHASHSEED'] = str(rng_seed)
torch.manual_seed(rng_seed)
torch.cuda.manual_seed(rng_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

num_species = 24
batch_size = 8

fft = 2048
hop = 512 
# According to research, standard sampling bitrate is 48khz. Seen in discussion of kaggle competition as well. 
sr = 48000
length = 10*sr
# ResNet50 input layer is 224 x 224 x 3, so I'm resizing the image to fit the first input dimension. 
mel_spec_dimensions = (224,224)

data_path = '../Data/'

df = pd.read_csv(data_path + 'train_tp.csv')

In [24]:
@pipeline_def
def spectogram_pipeline(df, idx, device='cpu'):
    
    rid = df.iloc[idx]['recording_id']
    wav, sr = librosa.load(data_path + 'train/' + rid + '.flac', sr=None)
    
    audio_data = np.array(wav, dtype=np.float64)
    
    audio = types.Constant(device=device, value=audio_data)
    spectrogram = fn.spectrogram(audio, device=device, nfft=fft, 
                                 window_length=length,
                                 window_step=hop)
    mel_spectrogram = fn.mel_filter_bank(spectrogram, sample_rate=sr, nfilter = 128, freq_high = 8000.0)
    mel_spectrogram_dB = fn.to_decibels(mel_spectrogram, multiplier = 10.0, cutoff_db = -80)
    return mel_spectrogram_dB


In [13]:

def show_spectrogram(spec, title, sr, hop_length, y_axis='log', x_axis='time'):
    librosa.display.specshow(spec, sr=sr, y_axis=y_axis, x_axis=x_axis, hop_length=hop_length)
    plt.title(title)
    plt.colorbar(format='%+2.0f dB')
    plt.tight_layout()
    plt.show()

In [22]:
os.system('nvidia-smi')
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

Thu Mar 10 17:17:24 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.39.01    Driver Version: 510.39.01    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:06:00.0 Off |                    0 |
| N/A   43C    P0   206W / 300W |  16072MiB / 16384MiB |    100%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  On   | 00000000:07:00.0 Off |                    0 |
| N/A   

In [26]:
pipe = spectogram_pipeline(df, 1, num_threads=3, batch_size=1)
pipe.build()
output = pipe.run()
mel_spec = np.array(outputs[0][0].as_cpu())
show_spectogram(mel_spec, "spec", sr, hop)

RuntimeError: Critical error in pipeline:
Exception in CPU stage: [/opt/dali/dali/pipeline/executor/executor.cc:62] Assert on "device_id_ == CPU_ONLY_DEVICE_ID" failed: Wrong device_id provided, it should be >= 0, or equal to CPU_ONLY_DEVICE_ID.
Stacktrace (8 entries):
[frame 0]: /opt/conda/lib/python3.8/site-packages/nvidia/dali/libdali.so(+0x7e0bf) [0x154d17b820bf]
[frame 1]: /opt/conda/lib/python3.8/site-packages/nvidia/dali/libdali.so(dali::Executor<dali::AOT_WS_Policy<dali::UniformQueuePolicy>, dali::UniformQueuePolicy>::RunCPUImpl()+0xf1e) [0x154d17bed5be]
[frame 2]: /opt/conda/lib/python3.8/site-packages/nvidia/dali/libdali.so(dali::Executor<dali::AOT_WS_Policy<dali::UniformQueuePolicy>, dali::UniformQueuePolicy>::RunCPU()+0xe) [0x154d17bed9ae]
[frame 3]: /opt/conda/lib/python3.8/site-packages/nvidia/dali/libdali.so(+0xa4951) [0x154d17ba8951]
[frame 4]: /opt/conda/lib/python3.8/site-packages/nvidia/dali/libdali.so(+0x119f9c) [0x154d17c1df9c]
[frame 5]: /opt/conda/lib/python3.8/s